In [37]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
import numpy as np
from typing import List, Callable
# import tensorflow as tf
# from tensorflow.keras import layers


# matplotlib.font_manager._rebuild()
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결
sns.set(font="Malgun Gothic",rc={"axes.unicode_minus":False}, style='whitegrid')

In [38]:
data_root = "./data/daily"

## 1. Data 준비

### 1) Weather data

In [39]:
# load each data
temp_df = pd.read_csv(f'{data_root}/Temperatures.csv', encoding='cp949')
precip_df = pd.read_csv(f'{data_root}/Precipitation.csv', encoding='cp949').fillna(0)
wind_df = pd.read_csv(f'{data_root}/Wind.csv', encoding='cp949')
humidity_df = pd.read_csv(f'{data_root}/Humidity.csv', encoding='cp949')
sunshine_df = pd.read_csv(f'{data_root}/Sunshine.csv', encoding='cp949')

# merge
weather_df = temp_df.merge(precip_df).merge(wind_df).merge(humidity_df).merge(sunshine_df)
weather_columns = ["평균기온", "강수량mm", "평균풍속ms", "평균습도rh","일조합"]

selected_weather_df = weather_df[['일시']+weather_columns]
display(selected_weather_df.head(3))

,일시,평균기온,강수량mm,평균풍속ms,평균습도rh,일조합
0,2012-01-01,-3.0,0.0,2.9,64.5,4.4
1,2012-01-02,-4.8,0.0,2.3,66.5,5.9
2,2012-01-03,-4.5,0.4,2.5,68.3,2.7


### 2) Stock data

In [40]:
company_name = "Samsung Electronics Co"
stock_price_df = pd.read_csv(f'./data/stock/{company_name}.csv')
stock_columns = ['Close']
selected_stock_df = stock_price_df[['Date'] + stock_columns]
display(selected_stock_df.head(3))

,Date,Close
0,2000-01-04,6110.0
1,2000-01-05,5580.0
2,2000-01-06,5620.0


### 3) Merge

In [41]:
stock_weather_df = selected_weather_df.rename(columns={"일시":"Date"}).merge(selected_stock_df)
display(stock_weather_df)

,Date,평균기온,강수량mm,평균풍속ms,평균습도rh,일조합,Close
0,2012-01-02,-4.8,0.0,2.3,66.5,5.9,21600.0
1,2012-01-03,-4.5,0.4,2.5,68.3,2.7,22100.0
2,2012-01-04,-7.4,0.0,3.1,55.4,8.1,21600.0
3,2012-01-05,-5.7,0.0,1.5,49.8,9.0,21100.0
4,2012-01-06,-2.8,0.0,2.5,42.9,8.4,20800.0
...,...,...,...,...,...,...,...
2553,2022-05-20,20.6,0.0,2.3,52.8,10.8,68000.0
2554,2022-05-23,23.0,0.0,2.3,63.5,10.4,67900.0
2555,2022-05-24,22.9,0.0,2.1,56.8,12.3,66500.0
2556,2022-05-25,21.0,3.5,2.6,66.6,8.1,66400.0


In [42]:
display(stock_weather_df.isnull().sum())
display(stock_weather_df[stock_weather_df['평균풍속ms'].isnull()])
display(stock_weather_df[stock_weather_df['일조합'].isnull()])
stock_weather_df = stock_weather_df.fillna(0)

Date      0
평균기온      0
강수량mm     0
평균풍속ms    2
평균습도rh    0
일조합       9
Close     0
dtype: int64

,Date,평균기온,강수량mm,평균풍속ms,평균습도rh,일조합,Close
1460,2017-12-05,-4.2,0.1,NaN,40.1,2.1,51260.0
1461,2017-12-06,0.2,1.2,NaN,72.6,7.7,50020.0


,Date,평균기온,강수량mm,평균풍속ms,평균습도rh,일조합,Close
1418,2017-09-28,19.2,0.0,2.8,49.1,NaN,51260.0
1422,2017-10-12,11.4,0.0,2.0,71.0,NaN,54800.0
1449,2017-11-20,-0.4,0.0,1.4,55.3,NaN,55200.0
1454,2017-11-27,2.5,0.0,1.3,53.0,NaN,52640.0
1457,2017-11-30,-2.4,0.0,3.3,26.8,NaN,50800.0
1488,2018-01-18,2.1,0.0,1.9,59.6,NaN,49900.0
1494,2018-01-26,-14.8,0.0,2.6,34.9,NaN,50780.0
1882,2019-08-28,26.1,0.0,1.9,66.2,NaN,44150.0
2217,2021-01-07,-14.5,0.0,4.1,49.9,NaN,82900.0


## 2. Training
### 1) RNN

In [43]:
import torch
import torch.nn as nn

# cfg
batch_size = 32
t_dim = 24  # todo: 변수명 변경 필요..
input_dim = len(weather_columns) + len(stock_columns)
epochs = 1
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [44]:
import torch
import torch.nn as nn

class StockModel(nn.Module):
    def __init__(self, input_dim, t_dim, h_c=32, num_layers=1):
        super(StockModel, self).__init__()
        self.input_dim = input_dim
        self.t_dim = t_dim

        self.rnn = nn.GRU(input_size=input_dim, hidden_size=h_c, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(in_features=h_c, out_features=1)

    def forward(self, x):
        output, hn = self.rnn(x)
        return self.fc(output[:, :, :]) # batch, last_cell, out_dim


### 2) Dataloader

In [45]:
from torch.utils.data import DataLoader, Dataset

class StockDataset(Dataset):
    def __init__(self, df, t_dim, label:List = None,):
        super(StockDataset, self).__init__()
        self.df = df
        self.t_dim = t_dim
        if label is None:
            label = df.columns
        self.label = label
        self.total_length = len(df) - (t_dim + 1)

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        x = self.df.loc[idx: idx+self.t_dim-1].values
        y = self.df.loc[idx+1: idx+self.t_dim][self.label].values
        return x, y

# preprocess
# log
stock_weather_df['Close'] = np.log(1+stock_weather_df['Close'])
train_df = stock_weather_df[stock_weather_df['Date'] < '2019-01-01'].drop(columns='Date')
test_df = stock_weather_df[stock_weather_df['Date'] >= '2019-01-01'].drop(columns='Date')

train_mean, train_std = train_df.mean(), train_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

display(train_df.head(3), test_df.head(3))

,평균기온,강수량mm,평균풍속ms,평균습도rh,일조합,Close
0,-1.620981,-0.276420,-0.192611,0.498315,-0.234437,-1.366412
1,-1.593744,-0.241892,0.036974,0.620731,-1.045259,-1.281495
2,-1.857033,-0.276420,0.725729,-0.256583,0.323003,-1.366412


,평균기온,강수량mm,평균풍속ms,평균습도rh,일조합,Close
1720,-1.630060,-0.27642,-0.881366,-1.113494,0.475032,0.802278
1721,-1.502955,-0.27642,-1.225743,-1.385530,0.475032,0.690485
1722,-1.285061,-0.27642,-1.455328,-0.249782,-0.741201,0.675651


In [46]:
def train(model, train_loader, criterion, optimizer, epochs=10, device=torch.device('cpu')):
    model = model.to(device)
    for epoch in range(epochs):
        running_loss = 0
        for step, (x, y) in enumerate(train_loader):
            x, y = x.float().to(device), y.float().to(device)
            optimizer.zero_grad()

            pred = model(x)
            loss =criterion(pred, y)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (step+1) % 100 == 0:
                print(f"{epoch+1} epoch : {step+1 :5d}step - loss={running_loss/(step+1)}")
        print(f"{epoch+1} epoch : {step+1 :5d}step - loss={running_loss/(step+1)}")

    return model

model = StockModel(input_dim, t_dim)
ds = StockDataset(train_df, t_dim=t_dim, label=['Close'])
dataloader = DataLoader(ds, batch_size=batch_size, shuffle=True)
criterion = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

model = train(model, dataloader, criterion, optimizer, epochs, device=device)

1 epoch :    53step - loss=0.1332336607969032


In [47]:
model.state_dict()


OrderedDict([('rnn.weight_ih_l0',
              tensor([[ 0.0403, -0.0559,  0.0478,  0.1070,  0.0973,  0.1667],
                      [ 0.1327,  0.1791,  0.1289, -0.1639,  0.1684, -0.0186],
                      [-0.0306,  0.0312,  0.0587,  0.1357,  0.1172, -0.1888],
                      [-0.0349,  0.0347,  0.1418,  0.1537, -0.1482,  0.2464],
                      [-0.0886, -0.1315, -0.1282, -0.0825, -0.0174,  0.0998],
                      [ 0.0170,  0.0545, -0.2015,  0.1503, -0.1320, -0.0265],
                      [ 0.1793, -0.1413,  0.0904, -0.0509, -0.1059,  0.1615],
                      [ 0.0293, -0.0856, -0.0953,  0.1284, -0.1187,  0.0754],
                      [-0.0586,  0.0478,  0.0633, -0.1566, -0.1365,  0.1135],
                      [ 0.1692, -0.0691,  0.0797,  0.1238,  0.0917, -0.0127],
                      [ 0.0653,  0.1295, -0.1646,  0.1241, -0.0254, -0.0811],
                      [ 0.1082, -0.1032, -0.0338, -0.0355,  0.0419,  0.1946],
                      [ 0.0933